#Imports

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#Widgets

In [0]:
dbutils.widgets.removeAll()
dbutils.widgets.text("catalog","catalog_ecommerce")
dbutils.widgets.text("schema_source", "bronze")
dbutils.widgets.text("schema_sink", "silver")

#Constantes

In [0]:
catalog         =  dbutils.widgets.get("catalog")
schema_source   =  dbutils.widgets.get("schema_source")
schema_sink     = dbutils.widgets.get("schema_sink")

#Preparacion

In [0]:
df_productos_bronze = spark.table(f"{catalog}.{schema_source}.productos")
df_categories = spark.table(f"{catalog}.{schema_source}.categories")
#df_productos_bronze.cache()
#df_categories.cache()

#Conversion

In [0]:
df_products_silver = df_productos_bronze \
    .filter(col("product_id").isNotNull()) \
    .join(df_categories, 
          df_productos_bronze.product_category_name == df_categories.product_category_name,
          "left") \
    .select(
        df_productos_bronze["*"],
        df_categories["product_category_name_english"].alias("category_english")
    )\
    .withColumn("category_english",
                when(col("category_english").isNull(), "Desconocido")
                .otherwise(trim(upper(col("category_english"))))) \
    .withColumn("ingestion_date", current_timestamp()) \
    .dropDuplicates(["product_id"])

#Guardar tabla

In [0]:
df_products_silver.write.mode("overwrite").saveAsTable(f"{catalog}.{schema_sink}.productos")